In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pyulog

Labelling Process

In [ ]:
import subprocess
import os
import pandas
import os
import numpy as np

for parent_path in os.listdir('/content/drive/Logs/'):

  for addition in ['Benign Flight', 'GPS Jamming', 'GPS Spoofing']:
    # if "PX4" not in parent_path:
    #   continue
    if addition not in os.listdir('/content/drive/Logs/' + parent_path + '/'):
      path = '/content/drive/Logs/' + parent_path + '/'
    else:
      path = '/content/drive/Logs/' + parent_path + '/' + addition + '/'
    print(path)

    filepath = ''
    os.chdir(path)

    for x in os.listdir("./"):
      print(x)
      if ".ulg" in x:
        filepath = x
        break
    if not os.path.isdir(path + 'CSVs'):
      os.mkdir(path + 'CSVs')
    os.chdir(path)
    subprocess.run(["ulog2csv", "-o", path + "CSVs/", path + filepath])

    #GPS
    csv_list = ['vehicle_attitude_0', 'global_position_0','gps_position_0', 'local_position_0']

    existing_csv_list = os.listdir(path + "CSVs/")
    print(existing_csv_list)

    final_df = pandas.DataFrame()
    final_df_gps = pandas.DataFrame()
    final_df_gpsekf = pandas.DataFrame()


    for csv in csv_list:
        for exist_csv in existing_csv_list:
            if csv in exist_csv:
                temp_df = pandas.read_csv(path + "CSVs/" + exist_csv)
                #print(temp_df)
                if len(final_df.columns) == 0:
                    final_df = temp_df
                else:
                    final_df = final_df.merge(temp_df, how='outer',left_on='timestamp', right_on='timestamp')
                break

    final_df = final_df.sort_values('timestamp')
    final_df = final_df.set_index('timestamp')
    final_df = final_df.interpolate(axis=0, method='linear', limit_direction='both')

    past = False
    for column in final_df.columns:
        if "timestamp" in column:
            if not past:
                past = True
                continue
            print(column)
            final_df = final_df.drop(columns=column)

    target_column = []

    print(final_df)

    #Labelling for GPS
    target_column = []
    target_lon, target_lat = 0, 0
    for index, row, in final_df[['lat_x', 'lon_x']].iterrows():
        if len(target_column) == 0:
            print(index, row)
            target_lat = row['lat_x']
            target_lon = row['lon_x']

        if (row['lat_x'] > target_lat + 0.03 or row['lat_x'] < target_lat - 0.03) or (row['lon_x'] > target_lon + 0.03 or row['lon_x'] < target_lon - 0.03):
            target_column.append("malicious")
        else:
            target_column.append("benign")
    final_df['label'] = target_column

    final_df = final_df.replace([np.inf, -np.inf], np.nan).dropna(how='any', axis=1)

    if not os.path.isdir(path + 'CSVs/Condensed/'):
      os.mkdir(path + 'CSVs/Condensed/')
    final_df.to_csv(path + 'CSVs/Condensed/{}.csv'.format(addition))

FileNotFoundError: ignored

Show most important features

In [ ]:

df_benign_flight = pd.read_csv( '/content/Drive/Logs/benign.csv')
df_malicious_flight = pd.read_csv('/content/Drive/Logs/malicious.csv')

df_benign_flight_train = df_benign_flight.drop(columns=['timestamp', 'label'], errors='ignore')

feature_names = df_benign_flight_train.columns
x = df_benign_flight_train.values
x = StandardScaler().fit_transform(x) # standardizing the features
train_pca_gps = PCA(.85)
train_pc_data = train_pca_gps.fit_transform(x)
df_benign_flight_train = pd.DataFrame(data=train_pc_data)

# get features with the most influence
n_pcs = train_pca_gps.components_.shape[0]
print("Total kept components: %s" % n_pcs)
most_important = [np.abs(train_pca_gps.components_[i]).argmax() for i in range(n_pcs)]
most_important_names = [feature_names[most_important[i]] for i in range(n_pcs)]
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}
important_features = pd.DataFrame(dic.items())
print("Most important features: %s" % important_features)

NameError: ignored